In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
import numpy as np
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
import re
from typing import List
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive
/content/drive/MyDrive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#import data
kindle=pd.read_csv("kindle_reviews.csv",usecols=['asin','overall','reviewText','reviewTime','summary'])
kindle = kindle.dropna()

In [ ]:
kindle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982596 entries, 0 to 982618
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   asin        982596 non-null  object
 1   overall     982596 non-null  int64 
 2   reviewText  982596 non-null  object
 3   reviewTime  982596 non-null  object
 4   summary     982596 non-null  object
dtypes: int64(1), object(4)
memory usage: 45.0+ MB


In [ ]:
#subset data to pick reviews after 1st of June 2014
kindle['reviewTime'] = pd.to_datetime(kindle['reviewTime'])
kindle2=kindle[kindle['reviewTime']>= '2014-06-01']

In [ ]:
kindle2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115093 entries, 6 to 982618
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   asin        115093 non-null  object        
 1   overall     115093 non-null  int64         
 2   reviewText  115093 non-null  object        
 3   reviewTime  115093 non-null  datetime64[ns]
 4   summary     115093 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 5.3+ MB


In [ ]:
#RESET INDEX 
kindle2=kindle2.reset_index()

# PRE-PROCESSING (STOPWORDS, PUNCTUATIONS AND LEMMATIZE)

In [ ]:
#STOPWORDS - create custom stopwords list

custom_stopwords=set(STOPWORDS)
to_add = ["Im", 'its']
# stops = ["not","doesn't","didn't","wouldn't","shouldn't","couldn't","very","too","really","so"]
stops = ["not","doesn't","didn't","wouldn't","shouldn't","couldn't"]
custom_stopwords = list(custom_stopwords - set(stops) | set(to_add))
# custom_stopwords = re.findall(r'\w+',str(custom_stopwords))

In [ ]:
## lemmatization
## https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


In [ ]:

#REMOVE PUNCTUATIONS

# kindle2["cleaned_reviews_punc"]=kindle2["reviewText"].apply(lambda x: str(' '.join(re.findall(r'\w+',str(nltk.word_tokenize(x))))))    
kindle2["cleaned_reviews_punc"]=kindle2["reviewText"].apply(lambda x: str(' '.join(re.findall(r'[A-Za-z]{3,}',str(nltk.word_tokenize(x))))))    


#REMOVE STOPWORDS
cleaned_reviews_list=[]
for review in kindle2["cleaned_reviews_punc"]:
    #print(review)
    words = nltk.word_tokenize(review)
    #print(words)
    new_words = []
    for word in words:
        if word.lower() in custom_stopwords:
            continue
        else:
          new_words.append(word.lower())
        cleaned_review = " ".join(new_words)
    cleaned_reviews_list.append(cleaned_review)

cleaned_reviews= pd.Series(cleaned_reviews_list)  

kindle2["cleaned_reviews_punc_stop"]=cleaned_reviews


#LEMMATIZE

kindle2["cleaned_reviews_lemm"]=kindle2["cleaned_reviews_punc_stop"].apply(lambda x: lemmatize_sentence(x))


In [ ]:
# ----- EXPORT CLEANED DATA TO CSV for use ---
kindle2.to_csv("kindle_cleaned_AA_final.csv")


In [2]:
kindle2=pd.read_csv("kindle_cleaned_AA_final.csv")

In [3]:
kindle2.head()

,Unnamed: 0,index,asin,overall,reviewText,reviewTime,summary,cleaned_reviews_punc,cleaned_reviews_punc_stop,cleaned_reviews_lemm
0,0,6,B000F83SZQ,4,I enjoyed this one tho I'm not sure why it's c...,2014-06-10,Nice old fashioned story,enjoyed this one tho not sure why called Amy B...,enjoyed tho not sure called amy brewster myste...,enjoy tho not sure call amy brewster mystery n...
1,1,55,B000FC26RI,4,I thought it would be instructive to compare t...,2014-06-21,Compare and contrast for a better understanding,thought would instructive compare this Charles...,thought instructive compare charles johnston t...,think instructive compare charles johnston tra...
2,2,104,B000HA4FKY,5,I live in a town adjacent a 'medium' sized cit...,2014-06-12,when I REALLY want to know!,live town adjacent medium sized city Massachus...,live town adjacent medium sized city massachus...,live town adjacent medium size city massachuse...
3,3,189,B000JMLBHU,4,A manual on desert island survival in its own ...,2014-06-29,A classic for a reason,manual desert island survival its own right th...,manual desert island survival right book havin...,manual desert island survival right book have ...
4,4,200,B000JMLBHU,5,The author had a great story line. Brought eac...,2014-06-15,The Mysterious Island,The author had great story line Brought each c...,author great story line brought character life...,author great story line bring character life i...


In [4]:
# SEE SPILT OF DATA BY OVERALL RATINGS
kindle2.groupby("overall")["overall"].count()

overall
1     2183
2     3164
3     9367
4    27310
5    73069
Name: overall, dtype: int64

#TOPIC MODELLING


### 1) LATENT SEMANTIC ANALYSIS(LSA)


#### We will create 3 models .

1) For reviews with ratings <3

2) For reviews with ratings between 3 and 4

3) For reviews with ratings equal to 5 

In [6]:
#TF-IDF for all 3 pots

trigram_vectorizer1= TfidfVectorizer(ngram_range=(3,3),min_df=3, max_df=0.6)
trigram_vectorizer2 = TfidfVectorizer(ngram_range=(3,3),min_df=3, max_df=0.6,max_features=2000)
trigram_vectorizer3 = TfidfVectorizer(ngram_range=(3,3),min_df=3, max_df=0.6,max_features=2000)


In [7]:
X_kindle_tri1, kindle_terms_tri1  = trigram_vectorizer1.fit_transform(kindle2[kindle2["overall"]<3].cleaned_reviews_lemm.values.astype('U'))\
                                                                                 , trigram_vectorizer1.get_feature_names_out()
X_kindle_tri2, kindle_terms_tri2  = trigram_vectorizer2.fit_transform(kindle2[(kindle2["overall"]>=3) & (kindle2["overall"]<5)].cleaned_reviews_lemm.values.astype('U'))\
                                                                                 , trigram_vectorizer2.get_feature_names_out()
X_kindle_tri3, kindle_terms_tri3  = trigram_vectorizer3.fit_transform(kindle2[kindle2["overall"]==5].cleaned_reviews_lemm.values.astype('U'))\
                                                                                 , trigram_vectorizer3.get_feature_names_out()

In [8]:
X_kindle_tri_idf1 = pd.DataFrame(X_kindle_tri1.toarray(), columns=kindle_terms_tri1)
X_kindle_tri_idf2 = pd.DataFrame(X_kindle_tri2.toarray(), columns=kindle_terms_tri2)
X_kindle_tri_idf3 = pd.DataFrame(X_kindle_tri3.toarray(), columns=kindle_terms_tri3)

In [ ]:
# X_kindle_tri_idf1.head()
# X_kindle_tri_idf2.head()
# X_kindle_tri_idf3.head()

In [9]:
# Function to get top tokens corresponding to each token

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 10):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
      print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)




In [10]:
# Function to get top documents corresponding to each topic

def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
  sorted_docs = W.argsort(axis=0)[::-1]
  top_docs = sorted_docs[:num_docs].T
  per_document_totals = W.sum(axis=1)
  for topic, top_documents_for_topic in enumerate(top_docs):
    print(f"Topic {topic}")
    for doc in top_documents_for_topic:
      score = W[doc][topic]
      percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
      print(f"{percent_about_topic}%", documents[doc])
    print("=" * 50)

In [11]:
#SVD DECOMPOSITION

from sklearn.decomposition import TruncatedSVD

# we need to select a K (the number of topics)
#svd = TruncatedSVD(n_components=K)


svd = TruncatedSVD(n_components=5)
U1 = svd.fit_transform(X_kindle_tri_idf1)
V_star1 = svd.components_

In [12]:
svd = TruncatedSVD(n_components=5)
U2 = svd.fit_transform(X_kindle_tri_idf2)
V_star2 = svd.components_

In [13]:
svd = TruncatedSVD(n_components=15)
U3 = svd.fit_transform(X_kindle_tri_idf3)
V_star3 = svd.components_

In [14]:
# TOP DOCUMENTS PER TOPIC (RATINGS 1 & 2)

get_top_tf_idf_tokens_for_topic(V_star1, X_kindle_tri_idf1.columns.tolist(), 5)

TOPIC 0

not recommend book (35.3%)

book not recommend (2.4%)

book main character (2.1%)

not like book (1.9%)

sorry not recommend (1.7%)

TOPIC 1

not cup tea (41.3%)

not worth time (6.1%)

want like book (3.2%)

cup tea not (2.7%)

waste time read (2.1%)

TOPIC 2

not like book (26.6%)

not finish book (20.2%)

story line not (2.1%)

finish book not (1.4%)

rarely not finish (1.4%)

TOPIC 3

not finish book (104.0%)

finish book not (7.5%)

rarely not finish (7.4%)

finish book read (5.3%)

not not finish (4.4%)

TOPIC 4

want like book (44.3%)

like book read (3.1%)

like book love (2.9%)

book sound good (1.5%)

save money not (1.5%)



In [15]:
# TOP TOKENS PER TOPIC (RATINGS 3 & 4)
get_top_tf_idf_tokens_for_topic(V_star2, X_kindle_tri_idf2.columns.tolist(), 5)

TOPIC 0

look forward read (13.8%)

exchange honest review (5.3%)

forward read book (3.3%)

book exchange honest (2.3%)

book look forward (1.8%)

TOPIC 1

exchange honest review (14.9%)

book exchange honest (6.4%)

copy book exchange (4.9%)

receive copy book (3.1%)

author exchange honest (3.1%)

TOPIC 2

look forward book (47.0%)

forward book series (11.4%)

book look forward (3.1%)

read look forward (3.0%)

story look forward (1.7%)

TOPIC 3

read book series (24.0%)

wait read book (7.2%)

forward read book (1.8%)

not read book (1.6%)

want read book (1.4%)

TOPIC 4

receive free copy (17.2%)

free copy book (15.4%)

wicked read review (12.4%)

read review team (11.7%)

read review wicked (11.2%)



In [16]:
# TOP TOKENS PER TOPIC (RATINGS=5)
get_top_tf_idf_tokens_for_topic(V_star3, X_kindle_tri_idf3.columns.tolist(), 5)

TOPIC 0

highly recommend book (33.2%)

book highly recommend (2.5%)

look forward read (1.6%)

read highly recommend (1.3%)

recommend book look (1.3%)

TOPIC 1

look forward read (23.3%)

forward read book (6.2%)

forward read author (3.2%)

book look forward (3.0%)

read book series (2.8%)

TOPIC 2

look forward book (28.1%)

forward book series (4.5%)

wait read book (2.9%)

book look forward (2.8%)

exchange honest review (2.5%)

TOPIC 3

wait read book (18.4%)

read book series (9.8%)

not wait read (5.9%)

exchange honest review (2.9%)

read book author (1.1%)

TOPIC 4

exchange honest review (16.6%)

book exchange honest (6.1%)

copy book exchange (4.4%)

author exchange honest (3.8%)

receive copy book (2.4%)

TOPIC 5

book highly recommend (51.0%)

highly recommend read (4.5%)

great book highly (4.2%)

read book highly (3.5%)

love book highly (2.8%)

TOPIC 6

read book series (53.3%)

forward read book (4.8%)

book series love (4.7%)

not w

In [17]:
# TOP DOCUMENTS PER TOPIC (RATINGS 1 and 2)
get_top_documents_for_each_topic(U1, kindle2[kindle2["overall"]<3]["reviewText"].tolist())

Topic 0
105.1% Did not like it at all, there was so many errors I had to keep stopping almost twice on the same page  THROUGHOUT THE ENTIRE BOOK and figure out what the story was about because sometimes she had to wipe her &#34;noise&#34; while crying (for example), and he was so freaking manipulative to her where he fires her so she'd be dependent on him and she in turns ends up living in her car with her 8 yo brother in the school parking lot! WTF! Will NOT be recommending this book to anyone.
105.1% This book contains some great illustrations.  However, the story is very unrealistic for my home.  What bothers me with this story is how passive the parents are.  They repeatedly ask Emma to clean her room which she never does.  This behavior would be considered disobedient and disrespectful in my home and would meet with a consequence enforced by the adults.  Because I don&#8217;t want my child exposed to media that says directly or indirectly it is ok to ignore adults, disobey adults 

In [18]:
# TOP DOCUMENTS PER TOPIC (RATINGS 3 and 4)
get_top_documents_for_each_topic(U2,kindle2[(kindle2["overall"]>=3) & (kindle2["overall"]<5)]["reviewText"].tolist())

Topic 0
288.1% Jin and his friends now inhabit my imagination as the characters are well developed and written. While this book was slightly less engaging than the first, it still kept my interest and I am looking forward to reading the next. In fact I am hoping for several more after that if the author gets busy!
288.1% This novel is a light-hearted adventure suitable for YA or even children. It has great lovable characters and a well thought out plot that is far from obvious. It tells the tale of a young and brave girl named Kira, who risks all to save her beloved father from his death sentence in a notorious prison. I found this book a charming read, light but with just the right hint of darkness to give the story some juice. I look forward to reading more of Katie&#8217;s books.
288.1% Fascinating tale of the publishing world. Affair with a married man, not so interesting. Difficult for me to get past the viewpoint of the wife, as I am one. I did get pulled into the life of Kate an

In [19]:
# TOP DOCUMENTS PER TOPIC (RATING 5)
get_top_documents_for_each_topic(U3,kindle2[kindle2["overall"]==5]["reviewText"].tolist())

Topic 0
136.4% I found this book helpful on many levels. As a mom of two elementary school students, as a business owner and entrepreneur, the tools in this concise, highly-enjoyable book gave me valuable insights for better utilizing my resources to nurture a culture of prosperity in running my days. A novice at understanding the principles of Feng Shui, I found this book to be well-written in terms of giving educating me on the five elements of Feng Shui and I was initially intrigued by this title because marrying what I knew of Feng Shui with wealth seemed a strange marriage to strike. But this book didn&#8217;t disappoint in any way.The author&#8217;s style is informative without being too technical, as well as engaging and attention-grabbing. Clear learning objectives are identified, so you know exactly what you&#8217;ll learn by reading this book! For the value this book brings in giving much food for thought to all those inclined to succeed and for teaching me several excellent 




## 2. BERT TOPIC MODEL

#### We will create 3 models .

1) For reviews with ratings <3

2) For reviews with ratings between 3 and 4

3) For reviews with ratings equal to 5 

In [ ]:
!pip install bertopic
from bertopic import BERTopic

In [ ]:
# create model1 (RATINGS=5)
 
model = BERTopic(verbose=True)
 
#convert to list 
docs = kindle2[kindle2["overall"]==5]["cleaned_reviews_lemm"].to_list()
 
# docs[1:10]
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/2284 [00:00<?, ?it/s]

2022-05-12 08:40:15,238 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-12 08:42:35,096 - BERTopic - Reduced dimensionality
2022-05-12 08:42:46,549 - BERTopic - Clustered reduced embeddings


In [ ]:
model.save("my_topics_model1")


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
BerTopic_model = BERTopic.load("my_topics_model1")

In [ ]:
BerTopic_model.get_topic_freq()

,Topic,Count
0,-1,41810
1,0,1003
2,1,659
3,2,521
4,3,432
...,...,...
589,603,10
588,602,10
587,601,10
586,605,10


In [ ]:
BerTopic_model.get_topic(1)

[('plot', 0.015684531324052387),
 ('development', 0.010811032300854726),
 ('character', 0.010541418767077763),
 ('storyline', 0.010378630765264544),
 ('line', 0.009148795271554763),
 ('believable', 0.006974180494066403),
 ('good', 0.0064682921230997436),
 ('pace', 0.00641059527327355),
 ('interest', 0.0063606671632928935),
 ('excellent', 0.006309829933736354)]

In [ ]:
BerTopic_model.get_topic_info(1)

,Topic,Count,Name
2,1,659,1_plot_development_character_storyline


In [ ]:
# BerTopic_model.get_representative_docs(topic=1)

In [ ]:
#VISUALIZE TOPICS
BerTopic_model.visualize_barchart(top_n_topics=20)

In [ ]:
# create model2 (POOR RATINGS)
 
model2 = BERTopic(verbose=True)
 
#convert to list 
docs = kindle2[kindle2["overall"]<3]["cleaned_reviews_lemm"].to_list()
 
# docs[1:10]
topics, probabilities = model2.fit_transform(docs)

Batches:   0%|          | 0/168 [00:00<?, ?it/s]

2022-05-12 08:46:38,176 - BERTopic - Transformed documents to Embeddings
2022-05-12 08:46:57,311 - BERTopic - Reduced dimensionality
2022-05-12 08:46:57,595 - BERTopic - Clustered reduced embeddings


In [ ]:
#SAVE MODEL
model2.save("my_topics_model2")

In [ ]:
model2.get_topic_freq()

,Topic,Count
0,0,5225
1,1,56
2,-1,36
3,2,30


In [ ]:
#VISUALIZE BARCHART
model2.visualize_barchart()

In [ ]:
model2.get_representative_docs(topic=0)

['review spoiler lily happily married stay home mom teenaged kid think marriage happy day catch husband charlie neighbor instant life fall apart confront let know saw okay wish shock want decide fact tell know wait lily make appointment consultation sexy divorce attorney eli option week yes exasperate sigh go month live house like wrong gather evidence try money leave lily eli build friendship sexual tension lily office work pay legal fee wait salon try desirable woman instead harry housewife lily meet old woman offer answer money problem know little prostitution lily think lose divorce idea charlie make figure nice house car prenup evidence indiscretion want charlie money hook option wait give terrible saw twist come mile away plus long moral high ground cheater keep reading sure lily confrontation charlie go epic wrong happen end little drama maybe take book seriously think suppose light fun tick',
 'think christian book miss see downloaded friend childhood adult run million dollar b

In [ ]:
# create model3 (RATINGS= 3 or 4)
 
model3 = BERTopic(verbose=True)
 
#convert to list 
docs = kindle2[(kindle2["overall"]>=3) & (kindle2["overall"]<5)]["cleaned_reviews_lemm"].to_list()
 
# docs[1:10]
topics, probabilities = model3.fit_transform(docs)

Batches:   0%|          | 0/1147 [00:00<?, ?it/s]

2022-05-12 09:04:44,071 - BERTopic - Transformed documents to Embeddings
2022-05-12 09:05:43,128 - BERTopic - Reduced dimensionality
2022-05-12 09:05:49,964 - BERTopic - Clustered reduced embeddings


In [ ]:
#SAVE MODEL
model3.save("my_topics_model3")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
model3.visualize_barchart(top_n_topics=15)

In [ ]:
model3.get_representative_docs(topic=0)

['review ramblings chick ramblingsfromthischick blogspot comfull exposure book independence fall series sara jane stone book read definitely enjoy story think write character interest easy relate book great quick easy read think story different read return serve country georgia take job work brother best friend nanny thing eric seek rush adrenaline alive jumping plane seek thrill know sex eric earth shatter adventure eric refuse touch desire eye act feel lose sister horrible accident eric take nephew nate focus make priority best friend ask little sister job eric help agree georgia determine friendship level matter time eric try avoid cross line time spend eric realizes georgia not girl refuse thrill eric georgia figure way past obstacle brother provide stable relationship nate thing keep pursuing future like georgia strong smart great role model proof woman great thing achieve success men dominate profession love soldier afraid tough situation able come home nanny nate provide care te